In [3]:
import os
import numpy as np

In [5]:
from scipy.io import loadmat
import six
def mat_to_data(path):
    mat = loadmat(path)
    names = mat['dataStruct'].dtype.names
    ndata = {n: mat['dataStruct'][n][0, 0] for n in names}
    for kk,vv in six.iteritems(ndata):
#         print(vv.shape)
        if vv.shape == (1,1):
            ndata[kk] = vv[0,0]
    return ndata

def get_label(infile):
    return infile.split(".")[-2][-1] == "0"

def test_mat_to_data(path = "./data/1_1_0.mat"):
    data = mat_to_data(path)
    label = get_label(path)

In [6]:
def simplegen(folder):
    """I_J_K.mat - the Jth training data segment 
    for the Ith patient (there are three patients)
    corresponding to the Kth class (K=0 for interictal, K=1 for preictal) ."""
    infiles = list(filter(lambda x: x.endswith("mat"), os.listdir(folder)))
    NUM_FILES = len(infiles)
    for nn, ff in enumerate(infiles):
        print(ff)
        ff = os.path.join(folder,ff)
        meta = [int(k) for k in ff.split("/")[-1].split(".")[0].split("_")[:2]]
        label = get_label(ff)
        data = mat_to_data(ff)
#         yield np.dstack(data["data"]).transpose(0,2,1), np.array([[label]])
        xx= (data["data"])
        piece_len=1000
        batch_size = xx.shape[0]//piece_len
        nchannels = xx.shape[1]
        yy = np.array(label)
        yield xx, yy, meta


def filegenchop(folder, piece_len=1000):
    gen = simplegen(folder)
    for nn, (xbig, ybig, meta) in enumerate(gen):
        piece_len=1000
        batch_size = xbig.shape[0]//piece_len
        nchannels = xbig.shape[1]
        xbig = np.reshape(xbig, (batch_size, piece_len, nchannels))
        for xx in xbig:
            yield xx, ybig, meta

1_1_0.mat
(1000, 16) True [1, 1]


In [7]:
import sqlite3

def chop_into_database( datadir,
        piece_len = 1000,
        tablename = "train1",
        ):
    
    dbname = "%s_piece_%u.db" % (tablename, piece_len)
    print("SAVING TO ", dbname)
    with sqlite3.connect(dbname) as conn:
        curs = conn.cursor()

        print("PURGING")
        qry = "DROP TABLE IF EXISTS %s" % tablename
        curs.execute(qry)

        print("CREATING")
        qry = """CREATE TABLE IF NOT EXISTS train1(
        id INT PRIMARY KEY,
        label INT,
        data BLOB,
        individual INT,
        segment INT
        )"""

        curs.execute(qry)

        insert_qry = "INSERT INTO %s (id, label, data, individual, segment) VALUES (?,?,?,?,?)"  % tablename
        gen = filegenchop(datadir, piece_len=piece_len)

        print("CHOPPING AND INSERTING")
        for id_, (xx, yy, meta) in enumerate(gen):
        #     print(xx.shape, yy, meta)
            label = bool(yy)
            blob = sqlite3.Binary(xx.tobytes())
            curs.execute(insert_qry, (id_, label, blob, meta[0], meta[1]))
    return dbname

In [17]:
tablename =  "train1"
piece_len = 1000
dbname = chop_into_database( datadir,
        piece_len = piece_len,
        tablename =tablename,
        )

SAVING TO  train1_piece_1000.db
CHOPPING AND INSERTING
1_1_0.mat
1_1_1.mat
1_2_0.mat
1_2_1.mat


In [13]:
with sqlite3.connect(dbname) as conn:
    curs = conn.cursor()
    curs.execute("SELECT COUNT(*) FROM %s" % tablename)
    print(curs.fetchone())

(960,)


In [14]:
conn.close()

In [15]:
def reconstruct_from_sql(row, nchannels = 16):
    row = list(row)
    row[2] = np.fromstring(row[2], dtype=np.dtype('<f4'),).reshape(-1, nchannels)
    return row

In [18]:
"TEST"
with sqlite3.connect(dbname) as conn:
    curs = conn.cursor()
    gen = filegenchop(datadir, piece_len=piece_len)
    xx, yy, meta = next(gen)

    curs.execute("SELECT * FROM %s" % tablename)
    out = reconstruct_from_sql(curs.fetchone())
    xx_reconstr = out[2]

    assert (xx_reconstr == xx).all()

1_1_0.mat


In [19]:
piece_len=1000
batch_size = xx.shape[0]//piece_len
nchannels = xx.shape[1]
# np.reshape(xx, (batch_size, piece_len, nchannels)).shape
# np.reshape(yy, (batch_size, piece_len)).shape
yy.shape

()

In [21]:
# (np.reshape(xx, (batch_size, piece_len, nchannels))[1] == xx[1000:2000,:]).all()

In [33]:
240000/400 

10.0

In [34]:
#downsampled:
120000 / 1000

120.0

In [35]:
10*60/120

5.0

imdb example:

    X (25000 , 80)
      (num_samples, seq_len)
     
eeg:

    X (num_samples, seq_len, embedding_dim)

In [29]:
input_ = mo.layers[0].input
la = mo.layers[-1]

# grads = K.gradients(K.mean(la.output), la.input)

loss = K.mean(layer_output[:, filter_index, :, :])

# normalization trick: we normalize the gradient
# grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)

# this function returns the loss and grads given the input picture
iterate = K.function([input_], [loss, grads])

NameError: name 'loss' is not defined

In [22]:
type(la.input)

theano.tensor.var.TensorVariable

In [9]:
from itertools import cycle
datadir = "data/"
BATCH_SIZE = 4
gen = cycle(filegen(datadir, BATCH_SIZE = BATCH_SIZE))
# print(type(gen))
# print(gen.next())

mo.fit_generator(gen, nb_worker=1,
                 nb_epoch=300, samples_per_epoch = BATCH_SIZE,
                 callbacks=[change_lr])

1_1_0.mat
Epoch 1/300
1_1_1.mat
1_2_0.mat
1_2_1.mat
4/4 [==============================] - 16s - loss: 2.0971 - acc: 0.5000
Epoch 2/300
4/4 [==============================] - 14s - loss: 8.5893 - acc: 0.5000
Epoch 3/300
4/4 [==============================] - 14s - loss: 8.5872 - acc: 0.5000
Epoch 4/300
4/4 [==============================] - 14s - loss: 8.5830 - acc: 0.5000
Epoch 5/300
4/4 [==============================] - 13s - loss: 8.5778 - acc: 0.5000
Epoch 6/300
4/4 [==============================] - 14s - loss: 8.5709 - acc: 0.5000
Epoch 7/300
4/4 [==============================] - 16s - loss: 8.5632 - acc: 0.5000
Epoch 8/300
4/4 [==============================] - 12s - loss: 8.5544 - acc: 0.5000
Epoch 9/300
4/4 [==============================] - 17s - loss: 8.5451 - acc: 0.5000
Epoch 10/300
4/4 [==============================] - 15s - loss: 8.5356 - acc: 0.5000
Epoch 11/300
4/4 [==============================] - 12s - loss: 8.5260 - acc: 0.5000
Epoch 12/300
4/4 [================

KeyboardInterrupt: 